In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
# Melakukan proses stemming (penggunaan kata dasar)
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
ps = PorterStemmer()

In [ ]:
# Mendownload daftar kata yang ada (vocabulary)
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Set the max columns to none

pd.set_option('display.max_columns', None)

In [ ]:
# Login Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_final = pd.read_csv('/content/gdrive/MyDrive/Youtube/Youtube/data_final.csv',  encoding='latin-1')
data_final.head()

Unnamed: 0                                            comment     label
0           0            santai tapi pasti sangat tega narasinya  negative
1           1  aamiin giliran pak ganjar menolak israel ramai...  negative
2           2  mbahmu masih ada saja orang dengki kayak agan ...  negative
3           3  mbahmu kalau bukan moncong putih pasti pemjila...  negative
4           4  amangnya siapa yang ngejaja mau merdeka silaka...  negative

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
# Label encoding untuk kolom 'label'
le = LabelEncoder()
data_final['label'] = le.fit_transform(data_final['label'])

# Split data menjadi train dan test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data_final['comment'], data_final['label'], test_size=0.2, random_state=42
)

In [ ]:
# Tokenisasi
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data)

# Sequencing dan padding
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

train_padded = pad_sequences(train_sequences, maxlen=200, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=200, padding='post', truncating='post')

In [ ]:
from keras.callbacks import ModelCheckpoint

# Definisi model
model = Sequential([
    Embedding(input_dim=5000, output_dim=16, input_length=200),
    tf.keras.layers.SpatialDropout1D(0.4),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(64)),
    Dense(64, activation = 'relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Menyusun ModelCheckpoint callback
checkpoint = ModelCheckpoint('youtube_v3.h5',
                            monitor='val_accuracy',
                            save_best_only=True,
                            mode='max',
                            verbose=1)

# Melatih model dengan menggunakan callback ModelCheckpoint
history = model.fit(train_padded, train_labels,
                    epochs=25,
                    validation_data=(test_padded, test_labels),
                    callbacks=[checkpoint])

Epoch 1/25
662/662 [==============================] - ETA: 0s - loss: 0.3804 - accuracy: 0.8299
Epoch 1: val_accuracy improved from -inf to 0.91313, saving model to youtube_v3.h5
662/662 [==============================] - 137s 201ms/step - loss: 0.3804 - accuracy: 0.8299 - val_loss: 0.2264 - val_accuracy: 0.9131
Epoch 2/25
  1/662 [..............................] - ETA: 1:36 - loss: 0.2903 - accuracy: 0.8125

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


662/662 [==============================] - ETA: 0s - loss: 0.2234 - accuracy: 0.9161
Epoch 2: val_accuracy improved from 0.91313 to 0.92691, saving model to youtube_v3.h5
662/662 [==============================] - 132s 199ms/step - loss: 0.2234 - accuracy: 0.9161 - val_loss: 0.1960 - val_accuracy: 0.9269
Epoch 3/25
662/662 [==============================] - ETA: 0s - loss: 0.1616 - accuracy: 0.9396
Epoch 3: val_accuracy improved from 0.92691 to 0.93144, saving model to youtube_v3.h5
662/662 [==============================] - 132s 199ms/step - loss: 0.1616 - accuracy: 0.9396 - val_loss: 0.1888 - val_accuracy: 0.9314
Epoch 4/25
662/662 [==============================] - ETA: 0s - loss: 0.1373 - accuracy: 0.9523
Epoch 4: val_accuracy did not improve from 0.93144
662/662 [==============================] - 132s 199ms/step - loss: 0.1373 - accuracy: 0.9523 - val_loss: 0.1865 - val_accuracy: 0.9311
Epoch 5/25
662/662 [==============================] - ETA: 0s - loss: 0.1206 - accuracy: 0.9568

In [ ]:
from keras.models import load_model
# Memuat model yang telah disimpan
loaded_model = model  # Ganti 'best_model.h5' dengan nama file model yang sesuai

# Melakukan prediksi pada data uji
predictions = loaded_model.predict(test_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

# Evaluasi
accuracy = accuracy_score(test_labels, predictions)
conf_matrix = confusion_matrix(test_labels, predictions)
classification_rep = classification_report(test_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


166/166 [==============================] - 12s 61ms/step
Accuracy: 0.9656279508970728
Confusion Matrix:
[[1914   74]
 [ 108 3199]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1988
           1       0.98      0.97      0.97      3307

    accuracy                           0.97      5295
   macro avg       0.96      0.97      0.96      5295
weighted avg       0.97      0.97      0.97      5295



In [ ]:
# Test sentence
test_sentence = "aplikasi lemot, emosi saya"
# Convert the test sentence to a sequence
test_sequence = tokenizer.texts_to_sequences([test_sentence])
# Pad the sequence
padded_test = pad_sequences(test_sequence,padding='post',maxlen=50)
#Get the label based on probability 1 if p >= 0.5 else
prediction= model.predict(padded_test)
#Convert probability to label
pred_label = 1 if prediction >= 0.5 else 0
# Display the result
print("Test sentence: ", test_sentence)
print("Predicted sentiment: ", "Positive" if pred_label >= 0.5 else "Negative")

1/1 [==============================] - 0s 88ms/step
Test sentence:  aplikasi lemot, emosi saya
Predicted sentiment:  Negative
